## Feature extraction
In this notebook an approach to preprocessing microphone data is explored. The idea is to group individual readings from a 8-channel audio recording into sequences longer sequences (for example of length 20). This will effectively create a list of matrices from the original data. Then, each matrix can be used to extract individual channel data and compute generalized cross-correlation (GCC-PHAT) between each pair of microphones.

#### Imports

In [1]:
import numpy as np
import pandas as pd
import os
from itertools import combinations
from statistics import mode

#### GCC-PHAT computation
Here the function for computing GCC is defined:

In [2]:
def gcc_phat(x_1, x_2):
    """
    Function that will compute the GCC-PHAT
    cross-correlation of two separate audio channels
    
    Returns:
        A 1-D GCC vector
    """
    
    # Fourier transforms of the two signals
    X_1 = np.fft.rfft(x_1)
    X_2 = np.fft.rfft(x_2)
    
    # GCC-PHAT = [X_1(f)X_2*(f)] / |X_1(f)X_2*(f)|
    # See http://www.xavieranguera.com/phdthesis/node92.html for reference
    top = X_1 * np.conj(X_2)
    bottom = abs(top)
    
    # Return the cross correlation
    return np.fft.irfft(top / bottom)

#### Creating observations
In the following code cell one can find a function that accepts a pandas dataframe containing rows of consecutive readings from a microphone array and transforms it into a list of GCC matrices.

In [28]:
def create_observations(dataframe, step=20):
    """
    Create list of observations from the pandas dataframe.
    Each observation will be a GCC matrix, where each row is a vector
    of GCC between a given pair of microphones.
    
    Returns: 
        a tuple of observations and their corresponding labels
    """
    
    # Lists of observations and labels that will be populated
    X = []
    y = []
    
    # Loop through the dataframe and take subframes
    for i in range(0, len(dataframe) - step + 1, step):
        sub_df = dataframe.iloc[i : i + step]
        
        # Extract the label as the most common label
        # from the subframe
        label = mode(sub_df['angle'])
        y.append(label)
        
        # Extract the observation from subframe by discarding the label column
        observation = np.array(sub_df.iloc[:, :-1])
        
        mic_pairs = combinations(dataframe.columns[:-1], r=2)
        
        # Initialize a transformed observation, that will be populated with GCC vectors
        # of the observation
        transformed_observation = []
        
        # Compute GCC for every pair of microphones
        for mic_1, mic_2 in mic_pairs:
            x_1 = observation[:, int(mic_1[-1]) - 1]
            x_2 = observation[:, int(mic_2[-1]) - 1]
            
            gcc = gcc_phat(x_1, x_2)
            
            # Add the GCC vector to the GCC matrix
            transformed_observation.append(gcc)
            
        # Add the GCC matrix to the list of observations
        X.append(transformed_observation)
        
    return X, y

#### Testing the function to create observations

In [29]:
path = os.path.join('..', 'training_data', 'angle_0_dist_50.csv')
df = pd.read_csv(path, dtype=np.int16)

X, y = create_observations(df)

print('X shape: ' + str(np.shape(X)))
print('y shape: ' + str(np.shape(y)))

D:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide


X shape: (3326, 15, 20)
y shape: (3326,)
